# Access live lightroom data 
* Load the getsql module  
* Make a connection to the lightroom DSN; this leaves an alias of LR to run the queries
* At some stage I will save the data so the notebook will work without a live connection to Lightroom data. Plotly had issues with the first attempt :-(

Light room uses the following tables, among others 
![lightroom Schema](./schema.png)

In [11]:
import-module GetSQL
Get-sql -Session LR -Connection DSN=LR  | ft -AutoSize


Data Source                                        Database State Connection String
-----------                                        -------- ----- -----------------
C:\Users\mcp\Pictures\Lightroom\Catalog-2-v12.lrca          Open  DSN=LR



In [2]:
#r "nuget: xplot.plotly.interactive"

Installed Packages xplot.plotly.interactive, 4.0.7

Loading extensions from `C:\Users\mcp\.nuget\packages\xplot.plotly.interactive\4.0.7\interactive-extensions\dotnet\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

# Load plotly support

In [3]:
Import-Module .\plotly.psm1 

# Get the 10 most used lenses and all the known apertures.    

In [4]:
$lensSQL      = "SELECT count(metadata.image) AS images, LensRef.value AS lensModel
                 FROM   AgharvestedExifMetadata metadata JOIN AgInternedExifLens LensRef ON LensRef.id_Local = metadata.lensRef
                 Group by LensRef.value
                 Order by Images Desc" 

$apertureSQL  = "SELECT DISTINCT Aperture FROM AgharvestedExifMetadata where Aperture > 0 ORDER BY Aperture " 

$apertureList  = lr $apertureSQL
$lensList      = lr $lensSQL | Select-Object -First 10 # TOP isn't supported in SQL Lite

31 row(s) returned
21 row(s) returned


# Get a row of data for each lens; 
By querying the number of images for each aperture for that lens, then    
for each known apperture put either 0 or 1 + log(number of images at that apperture) - scale is better as a log scale   
So we have a row per lens and each row has a number for every aperture.

In [5]:
$imagesSQL    = @"
SELECT    COUNT(metadata.image) AS images, 
          metadata.Aperture,  
          LensRef.value         AS lensModel
FROM      AgharvestedExifMetadata metadata 
JOIN      AgInternedExifLens LensRef 
ON        metadata.lensRef = LensRef.id_Local 
WHERE     metadata.Aperture > 0 

AND       lensModel = '{0}'

GROUP BY  metadata.Aperture,  LensRef.value
"@
$logImagecount = foreach ($l in $lensList) {
    $rowForLens    = @()
    $ImagesForLens = lr ($imagesSql -f $l.lensmodel)  
    foreach ($a in $apertureList) {
        $i = $ImagesForLens.Where({$_.Aperture -eq $a.aperture}).images
        if ($i) {$rowForLens += ([math]::log10($i) + 1) }
        else    {$rowForLens += 0 }
    } 
    @(,$rowForLens)
}

22 row(s) returned
19 row(s) returned
18 row(s) returned
16 row(s) returned
8 row(s) returned
16 row(s) returned
14 row(s) returned
16 row(s) returned
14 row(s) returned
1 row(s) returned


# Draw a heat map

In [6]:
#x Axis is f/ stops - convert Lightroom's "Aperture values" into f/ number  (f = Square-root of 2^A).  
#y Axis is lens names. Reduce names like "SMC Pentax-DA 18-250mm F3.5-6.3 ED AL [IF]", to "DA 18-250mm F3.5-6.3" 
#z is the log of numbers of images for that lens at that apperture
#Mark the color bar at the logs of  0,1,10,100 etc for that number of images 
 
plot -Layout @{margin= @{l = 250}} {
    Heatmap -x ($apertureList | ForEach-Object {"f/" + [math]::Round([math]::Sqrt([math]::Pow(2,$_.Aperture)),1)}) `
            -y ($lensList     | ForEach-Object {$_.lensModel -replace '(mm.*\d)\D*$' , '$1' -replace "^\s*(HD|SMC)\s*Pentax-", "" }) `
            -z  $logImagecount -ColorBar @{ tickvals = @(0,1,2,2.4,3,3.4); ticktext= @("0","1","10","25","100","250")}
}

<!DOCTYPE html>

# Do new queries for apertures and focal length(s) of each lens ... 

In [6]:
$sql = @"
SELECT   COUNT(metadata.image) As Images, 
         metadata.Aperture,  
         metadata.focalLength, 
         LensRef.value         AS lensModel
FROM     AgharvestedExifMetadata  metadata  
JOIN     AgInternedExifLens LensRef 
ON       metadata.lensRef = LensRef.id_Local
WHERE    metadata.Aperture    BETWEEN 1 AND 8.1 
AND      metadata.focalLength BETWEEN 9 AND 151

AND      LensRef.value = '{0}'

GROUP BY metadata.Aperture,  metadata.focalLength  
"@  

$threeWaySplit = @{}
foreach ($lens in $lensList.lensmodel) {
    $rows =  lr -q ($sql -f $lens) 
    if ($rows) {$threeWaySplit.$lens = $rows}
 }

# ... and scatter plot the results. 
Note, you can click on a series name in the legend to hide / show it. 

In [23]:

PLOT -Width 1250 -height 600 {
    foreach ($lens in $lensList.lensmodel) {
        $lensData = $threeWaySplit.$lens
        if (-not $lensdata ) {continue}
        Scatter -name ($lens -replace '(mm.*\d)\D*$' , '$1' -replace "^\s*(HD|SMC)\s*Pentax-", "" )`
                -Mode "markers" -Marker @{'Size'= foreach ($l in $lensData) {10 + 5*[math]::Log($l.images)}}`
                -x $(foreach ($l in $lensData) {[int](0 + $l.focalLength )})`
                -y $(foreach ($l in $lensData) {0 + [math]::Round([math]::Sqrt([math]::Pow(2,$l.aperture)),1) })
    }
}

<!DOCTYPE html>

In [7]:
#r "nuget:SandDance.InteractiveExtension" 

Installed Packages SandDance.InteractiveExtension, 1.0.100

Loading extension script from `C:\Users\mcp\.nuget\packages\sanddance.interactiveextension\1.0.100\interactive-extensions\dotnet\extension.dib`

Explore data visually using the SandDance Explorer . 
 This extension adds the ability to sort, filter, and visualize data using the SandDance Explorer . Use the ExploreWithSandDance() extension method with variables of type JsonElement , IEnumerable or IDataView to render the data explorer. 
 
 
 using Microsoft.Data.Analysis;
 using System.Collections.Generic;
 using Microsoft.ML;

 var dataFrame = DataFrame.LoadCsv("./Data.csv");

 dataFrame.ExploreWithSandDance().Display();
 
 
 To set the SandDance Explorer use the following code 
 
 
 DataExplorer.SetDefault ();

In [8]:
Update-TypeData -AppendPath .\notebook.Types.Ps1xml

In [18]:
$null = sql -Session LR  -OutputVariable Data -sql @"
SELECT   COUNT(metadata.image) As Images, 
         metadata.Aperture,  
         metadata.focalLength 
FROM     AgharvestedExifMetadata  metadata  
JOIN     AgInternedExifLens LensRef 
ON       metadata.lensRef = LensRef.id_Local
WHERE    metadata.Aperture    BETWEEN 1 AND 8.1 
AND      metadata.focalLength BETWEEN 9 AND 151

GROUP BY metadata.Aperture,  metadata.focalLength  
"@  


$data.ExploreWithSandDance()

340 row(s) returned


# Close the database connection 
(otherwise LightRoom can be locked out)

In [19]:
lr -Close